# To Do
1. Account for: paired (?) and library type (?)
4. Download experiments (runs and metadata) using SRA toolkit and organize data folder structure using metadata
5. Check indexer settings?

We're using os and glob to do bash script stuff.

In [9]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

#ref folder name, ref genomes go here
ref_folder = 'references'
#os.system('mkdir '+ref_folder)

#data folder name, samples and runs go here
data = 'data'
#os.system('mkdir '+data)

#csv output folder name
csv = 'Ex'
#os.system('mkdir '+csv)

# Organize Directories Using Metadata
1. Match runs to samples using metadata
2. Create folder for each sample name
3. Move each run into corresponding sample folder
4. Quality check to ensure that: reference genomes are in correct folder and data is in correct folder structure

data->samples->runs

Will do manually for now, but can think about automating this. 

# Building Index
Use salmon to build index. Download genomes from ensemble and store in refences folder. Will align to references in parallel. This is written to align to as many reference genomes/transcriptomes as are put in the references folder. 

In [10]:
#grabs stuff and returns a list of them-used throughout
def grab_ref(ref_folder1):
    files=glob.glob(ref_folder1+'/*')
    #print(files)
    return(files)

#builds Salmon index from given file names, puts indexes into folder, INDEX_references 
def salmon_index(genome):
    #os.system('salmon index -t '+str(genome)+' -i INDEX_'+str(genome))
    print('salmon '+ genome)
    return 

#call grab_ref() and salmon_index() to build index for each reference genome
for i in grab_ref(ref_folder):
    salmon_index(i)

salmon references/Pseudomonas_aeruginosa_UCBPP-PA14_109.ffn.gz
salmon references/Pseudomonas_aeruginosa_PAO1_107.ffn.gz


# Get Quants for Each Reference Genome on a Per Sample Basis
We want all of the runs from each sample to be fed into salmon at one time. This is currently robust to number of runs, but is not robust to library type or paired vs unpaired. If mulitple files, give -a and then a space seperated string of file names. Files must be of same library type and aligned to same transcriptome. Need to check salmon output path and need to see about paired reads.

In [24]:
#loops through indexes and samples and runs quant for each sample and each index. 
index_names =[] #will use this later
for y in glob.glob('INDEX_'+ref_folder+'/*'):
    print(y)
    index_name = y.replace('INDEX_references/','').replace('.ffn.gz','')
    print('Index: '+index_name)
    index_names.append(index_name)
    for j in grab_ref(data):
        #print(j)
        sample_name= j.replace('data/','')
        print('Sample name: '+sample_name)
        runs = grab_ref(j)
        #print(runs)
        output_name='quants/'+index_name+'/'+sample_name+'_quant'
        print('Quant output:'+output_name)
        if len(runs)<1:
            print('Oops, no runs for '+sample_name)
        if len(runs)==1:
            print('One run for '+j)
            os.system('salmon quant -i '+y+' -l A -r '+runs[0]+' -p 8 --validateMappings -o '+output_name)
        if len(runs)>1:
            print('Multiple runs for '+sample_name)
            input_name='-r'
            for z in np.arange(0, len(runs)):
                if "R1" in runs[z]:
                    input_name+=' '+runs[z]
            for z in np.arange(0, len(runs)):
                if "R2" in runs[z]:
                    print('make reverse: '+runs[z])
                    os.system('gzip -d '+runs[z])
                    os.system("fastx_reverse_complement -z -i "+runs[z]+" -o "+runs[z]+"_R1")
            for z in np.arange(0, len(runs)):
                if "R1" in runs[0]:
                    os.system('gzip -d'+runs[0])
                    input_name+= ' '+runs[0]
                if "R1" not in runs[z]:
                    print('')
                    #os.system('rm '+runs[z])
            print('Salmon Input: '+input_name)
            #os.system('salmon quant -i '+y+' -l A '+input_name+' -p 8 --validateMappings -o '+output_name)

INDEX_references/Pseudomonas_aeruginosa_UCBPP-PA14_109.ffn.gz
Index: Pseudomonas_aeruginosa_UCBPP-PA14_109
Sample name: GD1-45169164-WT_rep1.3
Quant output:quants/Pseudomonas_aeruginosa_UCBPP-PA14_109/GD1-45169164-WT_rep1.3_quant
One run for data/GD1-45169164-WT_rep1.3
INDEX_references/Pseudomonas_aeruginosa_PAO1_107.ffn.gz
Index: Pseudomonas_aeruginosa_PAO1_107
Sample name: GD1-45169164-WT_rep1.3
Quant output:quants/Pseudomonas_aeruginosa_PAO1_107/GD1-45169164-WT_rep1.3_quant
One run for data/GD1-45169164-WT_rep1.3


# Storing Output

Similar to running the quants, for each reference genome, we're going to grab the quants on a sample by sample basis and construct a data frame for each reference genome. The dataframes will be stored in a list. 

Not sure if row names are robust. I think they are, but I have not thought hard about it. 

This will eatup ram (?) when implemented. Instead of storing the data from each sample in a dataframe, it should be appended to a csv file? Shouldn't be too hard to do, but will effect everything downstream. 

In [25]:
#loop through indexes and build dataframe of output for each one
df_list = []
for i in index_names:
    glob_list = []
    replace = 'quants/'+i+'/'
    #print(i)
    for z in grab_ref('quants/'+i):
        #print(z)
        for file in glob.glob(z+'/*.sf'):
            glob_list.append(file)
            #print(file)
    expression_df = pd.DataFrame(
    pd.read_csv(file, sep="\t", index_col=0)["TPM"].
        rename(file.replace(replace, '').replace('_quant/quant.sf',''))
    for file in glob_list)
    df_list.append(expression_df)
    expression_df.to_csv(csv+'/aligned_to_'+i, sep='\t')
print(df_list)

[Name                    PGD1650835  PGD1650837  PGD1650839  PGD1650841  \
GD9-45172138_pstB_rep2   35.277179   57.746692   25.622426   23.914272   
GD1-45169164-WT_rep1     49.944022   83.272098   37.949438   33.438258   
GD3-45171150-pstB_rep1   36.860554   54.972097   28.366097   25.248504   
SAMN02691047            181.071153  256.061143  148.308933  204.088772   
GD7-45169169_WT_rep2     54.039770   86.087977   35.995566   32.338511   
GD1-45169164-WT_rep1.3   49.420548   80.185449   37.806049   33.567150   

Name                    PGD1650843  PGD1650845  PGD1650847  PGD1650849  \
GD9-45172138_pstB_rep2    7.278564   12.579428  100.374052    6.596169   
GD1-45169164-WT_rep1     12.534982   20.219347  105.354902    9.833318   
GD3-45171150-pstB_rep1    8.525593   18.903904   81.690075    6.637134   
SAMN02691047             80.336943   83.965550   10.125983   57.898305   
GD7-45169169_WT_rep2     12.915141   17.952131   93.988283   11.059103   
GD1-45169164-WT_rep1.3   12.775361  